# OpenAI Function Calling In LangChain

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

**Note**: The next cell is expected to fail.

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [22]:
WeatherSearch.schema()

{'description': 'Call this with an airport code to get the weather at that airport',
 'properties': {'airport_code': {'description': 'airport code to get weather for',
   'title': 'Airport Code',
   'type': 'string'}},
 'required': ['airport_code'],
 'title': 'WeatherSearch',
 'type': 'object'}

In [21]:
schema = WeatherSearch.schema()
schema.pop("definitions", None)
schema

{'description': 'Call this with an airport code to get the weather at that airport',
 'properties': {'airport_code': {'description': 'airport code to get weather for',
   'title': 'Airport Code',
   'type': 'string'}},
 'required': ['airport_code'],
 'title': 'WeatherSearch',
 'type': 'object'}

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [23]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/Users/hungryfoolish/anaconda3/envs/agent310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [24]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

**Note**: The next cell is expected to generate an error.

In [26]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [27]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [28]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [37]:
from langchain.chat_models import ChatOpenAI

In [38]:
model = ChatOpenAI()

/Users/hungryfoolish/anaconda3/envs/agent310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [39]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-988e55d7-9791-4575-b3b1-55d13b0d76ff-0')

In [40]:
model_with_function = model.bind(functions=[weather_function])

In [41]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8da0ac2c-146e-4ce5-81e0-154420625d63-0')

### Ollama function calling

In [42]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions
oll = OllamaFunctions(model="mistral", format="json")

In [45]:
oll_bind = oll.bind(tools=[weather_function])

In [46]:
oll_bind.invoke("what is the weather in sf?")

AIMessage(content='The weather in San Francisco is currently not available through this tool. You may want to check a weather forecasting service for accurate and up-to-date information.', id='run-238480eb-0ce0-4366-990f-b7ca31021bd0-0')

it fails to call the function if we just bind the function to the model. passing function_call to force llm to use function calling then works

In [56]:
oll_bind_force = oll.bind_tools(
    tools=[{'name': 'WeatherSearch',
            'description': 'Call this with an airport code to get the weather at that airport',
            'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
                'type': 'string'}},
            'required': ['airport_code'],
            'type': 'object'}}], 
    function_call={'name': "WeatherSearch"})
oll_bind_force.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"properties": {"airport_code": {"description": "airport code for San Francisco", "type": "string", "value": "SFO"}}, "required": ["airport_code"], "type": "object"}'}}, id='run-42fb769f-9aa5-4400-808a-bf5ca90e11ce-0')

## Forcing it to use a function

We can force the model to use a function

In [ ]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [57]:
from langchain.prompts import ChatPromptTemplate

In [58]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [59]:
chain = prompt | oll_bind_force

In [60]:
chain.invoke({"input": "what is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"airport_code": "SFO"}'}}, id='run-d91b24ba-498a-462d-908a-dad868a1e913-0')

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [61]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [62]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [63]:
model_with_functions = model.bind(functions=functions)

In [64]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-62c07e62-b517-40d7-aad0-3217fb57a1c8-0')

In [65]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"taylor swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 118, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-8100f79c-9212-429c-9a59-6ceada0d19b1-0')

In [66]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 111, 'total_tokens': 121}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7ad9452c-2125-450f-8297-3bd20563f97a-0')

### choosing function to call with Ollama Mistral

In [67]:
oll_with_functions = oll.bind(functions=functions)

In [68]:
oll_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': '{"properties": {"airport_code": "SFO"}, "required": ["airport_code"], "type": "object"}'}}, id='run-16a0ae1b-b88c-4b92-9c90-20a8f2304318-0')

In [69]:
oll_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"artist_name": "Taylor Swift", "n": 3}'}}, id='run-a37b04bb-54bc-4240-8a03-b03096454948-0')

In [70]:
oll_with_functions.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{"properties": {"artist_name": "Ed Sheeran", "n": 5}}'}}, id='run-3a6b28f9-be87-4dfc-8c99-62eeb6a2d263-0')

The multi model Mistral is able to choose the right function, but it suffers from catastrophic forgetting so when we ask it a NLP question, it continues to force itself to call a function!